# Editing pretrained models


In [1]:


import keras
import numpy as np
import os
import numpy as np
import keras
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense


Using TensorFlow backend.


### Load pretrained network
This is hopefully a network that has been trained on a huge dataset. Only the convolution layers of this network will be used. 

In [2]:
m = keras.models.load_model('/home/wroscoe/d2/models/all.h5')
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
img_in (InputLayer)              (None, 120, 160, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 58, 78, 24)    1824        img_in[0][0]                     
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 27, 37, 32)    19232       conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 12, 17, 64)    51264       conv2d_2[0][0]                   
___________________________________________________________________________________________

When loaded Every layer except the top layer is trainable. 

In [3]:
[(l.name, l.trainable) for l in m.layers]

[('img_in', False),
 ('conv2d_1', True),
 ('conv2d_2', True),
 ('conv2d_3', True),
 ('conv2d_4', True),
 ('conv2d_5', True),
 ('flattened', True),
 ('dense_1', True),
 ('dropout_1', True),
 ('dense_2', True),
 ('dropout_2', True),
 ('angle_out', True),
 ('throttle_out', True)]

### Make all the layers of the loaded network NOT trainable.

In [4]:
#make all layers of the loaded network trainable
for l in m.layers:
    l.trainable = False

### Splice a tail to the network that is initialized (random) and trainable.

In [5]:
def add_tail_network(start_tensor, end_tensor):
    #x = Flatten(name='flattened')(x)                                        # Flatten to 1D (Fully connected)
    x = Dense(100, activation='relu', name="tail_dense_1")(end_tensor)                                    # Classify the data into 100 features, make all negatives 0
    x = Dropout(.1)(x)                                                      # Randomly drop out (turn off) 10% of the neurons (Prevent overfitting)
    x = Dense(50, activation='relu')(x)                                     # Classify the data into 50 features, make all negatives 0
    x = Dropout(.1)(x)                                                      # Randomly drop out 10% of the neurons (Prevent overfitting)
    #categorical output of the angle
    angle_out = Dense(15, activation='softmax', name='angle_out')(x)        # Connect every input with every output and output 15 hidden units. Use Softmax to give percentage. 15 categories and find best one based off percentage 0.0-1.0
    
    #continous output of throttle
    throttle_out = Dense(1, activation='relu', name='throttle_out')(x)      # Reduce to 1 number, Positive number only
    
    model = Model(inputs=[start_tensor], outputs=[angle_out, throttle_out])
    model.compile(optimizer='rmsprop',
                  loss={'angle_out': 'categorical_crossentropy', 
                        'throttle_out': 'mean_absolute_error'},
                  loss_weights={'angle_out': 0.9, 'throttle_out': .001})

    return model

In [6]:
m2 = add_tail_network(m.get_layer('img_in').input, m.get_layer('flattened').output)

#check that the layer "tail_dense_1 is in our new model.
m2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
img_in (InputLayer)              (None, 120, 160, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 58, 78, 24)    1824        img_in[0][0]                     
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 27, 37, 32)    19232       conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 12, 17, 64)    51264       conv2d_2[0][0]                   
___________________________________________________________________________________________

Here you can see the tail has been added because the layer `tail_dense_1` has been added

### Test the model changed as expected.

In [7]:
#weights of convolution layers are the same as saved model.
np.array_equal( m.layers[1].get_weights()[0], m2.layers[1].get_weights()[0])

True

In [8]:
#weights of dense layers in new model are different. 
np.array_equal( m.layers[-6].get_weights()[0], m2.layers[-6].get_weights()[0])

False

In [9]:
## convolution layers are NOT trainable
m2.layers[1].trainable

False

In [10]:
## dense layers are trainable
m2.layers[-6].trainable

True

### Save new model so it can be loaded later. 

In [11]:
template_model_path = '/home/wroscoe/models/vision_only.h5'
m2.save(template_model_path)

### Load template model and check that its the same. 

In [12]:
m3 = keras.models.load_model(v)

In [13]:
[(l.name, l.trainable) for l in m3.layers]

[('img_in', False),
 ('conv2d_1', False),
 ('conv2d_2', False),
 ('conv2d_3', False),
 ('conv2d_4', False),
 ('conv2d_5', False),
 ('flattened', False),
 ('tail_dense_1', True),
 ('dropout_1', True),
 ('dense_1', True),
 ('dropout_2', True),
 ('angle_out', True),
 ('throttle_out', True)]

###  Train on new data. 

In [19]:
import donkeycar as dk
from donkeycar.parts.keras import KerasCategorical
from donkeycar.parts.datastore import TubGroup

In [20]:
cfg = dk.config.load_config('/home/wroscoe/d2/config.py')

loading config file: /home/wroscoe/d2/config.py
config loaded


In [25]:
tub_names = ','.join(['/home/wroscoe/d2/data/tub_1_17-11-18/'])

In [33]:

X_keys = ['cam/image_array']
y_keys = ['user/angle', 'user/throttle']

def rt(record):
    record['user/angle'] = dk.utils.linear_bin(record['user/angle'])
    return record

kl = KerasCategorical()
kl.load(template_model_path)
print('tub_names', tub_names)
if not tub_names:
    tub_names = os.path.join(cfg.DATA_PATH, '*')
tubgroup = TubGroup(tub_names)
train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys, record_transform=rt,
                                                batch_size=cfg.BATCH_SIZE,
                                                train_frac=cfg.TRAIN_TEST_SPLIT)



tub_names /home/wroscoe/d2/data/tub_1_17-11-18/
TubGroup:tubpaths: ['/home/wroscoe/d2/data/tub_1_17-11-18']
path_in_tub: /home/wroscoe/d2/data/tub_1_17-11-18
Tub exists: /home/wroscoe/d2/data/tub_1_17-11-18
joining the tubs 6539 records together. This could take 0 minutes.


In [34]:
model_name = 'vision_test1.h5'

In [35]:
model_path = os.path.expanduser(model_name)

total_records = len(tubgroup.df)
total_train = int(total_records * cfg.TRAIN_TEST_SPLIT)
total_val = total_records - total_train
print('train: %d, validation: %d' % (total_train, total_val))
steps_per_epoch = total_train // cfg.BATCH_SIZE
print('steps_per_epoch', steps_per_epoch)

kl.train(train_gen,
         val_gen,
         saved_model_path=model_path,
         steps=steps_per_epoch,
         train_split=cfg.TRAIN_TEST_SPLIT)

train: 5231, validation: 1308
steps_per_epoch 40
Epoch 1/100
40/40 [==============================] - 25s - loss: 8.1766 - angle_out_loss: 9.0835 - throttle_out_loss: 1.4506 - val_loss: 7.1082 - val_angle_out_loss: 7.8972 - val_throttle_out_loss: 0.7132
Epoch 2/100
40/40 [==============================] - 30s - loss: 6.4571 - angle_out_loss: 7.1736 - throttle_out_loss: 0.8091 - val_loss: 6.2865 - val_angle_out_loss: 6.9843 - val_throttle_out_loss: 0.6189
Epoch 3/100
40/40 [==============================] - 29s - loss: 5.7360 - angle_out_loss: 6.3727 - throttle_out_loss: 0.6511 - val_loss: 5.6107 - val_angle_out_loss: 6.2334 - val_throttle_out_loss: 0.6148
Epoch 4/100
40/40 [==============================] - 24s - loss: 5.0216 - angle_out_loss: 5.5788 - throttle_out_loss: 0.6307 - val_loss: 5.2850 - val_angle_out_loss: 5.8716 - val_throttle_out_loss: 0.6154
Epoch 5/100
40/40 [==============================] - 24s - loss: 4.5809 - angle_out_loss: 5.0892 - throttle_out_loss: 0.6413 - val_

40/40 [==============================] - 25s - loss: 0.3986 - angle_out_loss: 0.4421 - throttle_out_loss: 0.6213 - val_loss: 1.0624 - val_angle_out_loss: 1.1797 - val_throttle_out_loss: 0.6217
Epoch 22/100
40/40 [==============================] - 24s - loss: 0.3473 - angle_out_loss: 0.3852 - throttle_out_loss: 0.6220 - val_loss: 1.2885 - val_angle_out_loss: 1.4310 - val_throttle_out_loss: 0.6184
Epoch 23/100
40/40 [==============================] - 24s - loss: 0.3328 - angle_out_loss: 0.3691 - throttle_out_loss: 0.6210 - val_loss: 1.1915 - val_angle_out_loss: 1.3232 - val_throttle_out_loss: 0.6180
Epoch 24/100
40/40 [==============================] - 24s - loss: 0.2891 - angle_out_loss: 0.3205 - throttle_out_loss: 0.6222 - val_loss: 1.2449 - val_angle_out_loss: 1.3825 - val_throttle_out_loss: 0.6177
Epoch 25/100
40/40 [==============================] - 24s - loss: 0.2850 - angle_out_loss: 0.3160 - throttle_out_loss: 0.6213 - val_loss: 1.1797 - val_angle_out_loss: 1.3101 - val_throttle_